In [22]:
import numpy as np
from collections import Counter

In [23]:
def HEntropy(l):
    length = len(l)
    cnt = Counter(l)
    
    ent = 0
    for cl in cnt.values():
        p = cl / length
        l2 = np.log2(p)
        it = -p * l2
        ent += it
    
    return ent

In [24]:
def HGini(l):
    length = len(l)
    cnt = Counter(l)
    
    gini = 0
    for cl in cnt.values():
        p_1 = cl / length
        p_2 = (1 - p_1)
        it = p_1 * p_2
        gini += it
    
    return gini

In [25]:
def IG(H, l, i):
    left_l = l[:i]
    right_l = l[i:]
    return H(l) - (len(left_l) / len(l)) * H(left_l) - (len(right_l) / len(l)) * H(right_l)

Нужен класс, чтобы понятно было попадет ли в левую часть разбиения. Для этго воспользуемся функицей splitter

node корень дерева

In [26]:
import pandas as pd

class Splitter:
   
    def __init__(self, attr_name, threshold_value):
        self.attr_name = attr_name
        self.threshold_value = threshold_value
        
    def split(self, item):
        return item[self.attr_name] < self.threshold_value
    
    def __str__(self):
        return f'{self.attr_name} < {self.threshold_value}'
        
class Node:
    def __init__(self, splitter, left_node, right_node):
        self.splitter = splitter
        self.left_node = left_node
        self.right_node = right_node

    def evaluate(self, item):
        if self.splitter.split(item):
            return self.left_node.evaluate(item)
        else:
            return self.right_node.evaluate(item)
        
class Leaf:
    def __init__(self, clazz):
        self.clazz = clazz
        
    def evaluate(self, item):
        return self.clazz

In [27]:
def get_best_splitter(training_df, attr_name):
    
    sorted_df = training_df.sort_values(attr_name)
    clazzes = sorted_df['clazz'].values
    max_ig = 0
    best_split_i = 0 
    for i in range(len(sorted_df)):
        ig = IG(HEntropy, clazzes, i)
        if ig > max_ig:
            max_ig = ig
            best_split_i = i
             
    best_split_value = sorted_df.iloc[best_split_i][attr_name]
    best_splitter = Splitter(attr_name, best_split_value)
    return max_ig, best_splitter


def build_tree(training_df, excluded_attrs=None):
    need_for_split = len(training_df['clazz'].unique()) > 1
    if not need_for_split:
        clazz = training_df.iloc[0]['clazz']
        return Leaf(clazz)
    
    if excluded_attrs == None:
        excluded_attrs = set(['clazz'])

    x_columns_names = [column for column in training_df.columns if column not in excluded_attrs]
    
    max_ig = 0
    best_splitter = None
    for attr_name in x_columns_names:
        ig, splitter = get_best_splitter(training_df, attr_name)
        if ig > max_ig:
            max_ig = ig
            best_splitter = splitter
    
    left_df = training_df[training_df.apply(best_splitter.split, axis=1)]
    right_df = training_df[~training_df.apply(best_splitter.split, axis=1)]
    
    if len(left_df) == 0 or len(right_df) == 0:
        excluded_attrs.add(best_splitter.attr_name)
        if(len(excluded_attrs) == len(training_df.columns)):
            raise Exception
        return build_tree(training_df, excluded_attrs)
    
    left_node = build_tree(left_df)
    right_node = build_tree(right_df)
    
    return Node(best_splitter, left_node, right_node)

In [28]:
from sklearn.datasets import load_iris

In [29]:
iris = load_iris()

training_df = pd.DataFrame(iris.data, columns=iris.feature_names)
training_df['clazz'] = pd.DataFrame(iris.target)

tree = build_tree(training_df)
tree

In [30]:
iris

{'data': array([[5.1, 3.5, 1.4, 0.2],
        [4.9, 3. , 1.4, 0.2],
        [4.7, 3.2, 1.3, 0.2],
        [4.6, 3.1, 1.5, 0.2],
        [5. , 3.6, 1.4, 0.2],
        [5.4, 3.9, 1.7, 0.4],
        [4.6, 3.4, 1.4, 0.3],
        [5. , 3.4, 1.5, 0.2],
        [4.4, 2.9, 1.4, 0.2],
        [4.9, 3.1, 1.5, 0.1],
        [5.4, 3.7, 1.5, 0.2],
        [4.8, 3.4, 1.6, 0.2],
        [4.8, 3. , 1.4, 0.1],
        [4.3, 3. , 1.1, 0.1],
        [5.8, 4. , 1.2, 0.2],
        [5.7, 4.4, 1.5, 0.4],
        [5.4, 3.9, 1.3, 0.4],
        [5.1, 3.5, 1.4, 0.3],
        [5.7, 3.8, 1.7, 0.3],
        [5.1, 3.8, 1.5, 0.3],
        [5.4, 3.4, 1.7, 0.2],
        [5.1, 3.7, 1.5, 0.4],
        [4.6, 3.6, 1. , 0.2],
        [5.1, 3.3, 1.7, 0.5],
        [4.8, 3.4, 1.9, 0.2],
        [5. , 3. , 1.6, 0.2],
        [5. , 3.4, 1.6, 0.4],
        [5.2, 3.5, 1.5, 0.2],
        [5.2, 3.4, 1.4, 0.2],
        [4.7, 3.2, 1.6, 0.2],
        [4.8, 3.1, 1.6, 0.2],
        [5.4, 3.4, 1.5, 0.4],
        [5.2, 4.1, 1.5, 0.1],
  

In [31]:
def print_tree(node, indentation=''):
    print(indentation + "if " + str(node.splitter)+ ':')
    
    if isinstance(node.left_node, Node):
        print_tree(node.left_node, indentation + "\t")
    else:
        print(indentation + f"\tclass = {node.left_node.clazz}")
        
    print(indentation + f"else:")
    
    if isinstance(node.right_node, Node):
        print_tree(node.right_node, indentation + "\t")
    else:
        print(indentation + f"\tclass = {node.right_node.clazz}")

print_tree(tree)

if petal length (cm) < 3.0:
	class = 0.0
else:
	if petal width (cm) < 1.8:
		if petal length (cm) < 5.0:
			if petal width (cm) < 1.7:
				class = 1.0
			else:
				class = 2.0
		else:
			if petal length (cm) < 5.1:
				if sepal length (cm) < 6.7:
					class = 2.0
				else:
					class = 1.0
			else:
				if sepal length (cm) < 6.1:
					class = 1.0
				else:
					class = 2.0
	else:
		if sepal length (cm) < 5.9:
			class = 2.0
		else:
			if sepal width (cm) < 3.2:
				class = 2.0
			else:
				if sepal length (cm) < 6.2:
					class = 1.0
				else:
					class = 2.0


Проверяем на обученных данных

In [32]:
training_df['guess'] = training_df.apply(tree.evaluate, axis=1)
right_guesses = training_df[training_df['guess'] == training_df['clazz']]
accuracy = len(right_guesses)/len(training_df)
print(f'accuracy = {accuracy}')

accuracy = 1.0
